In [1]:
# from minicons import scorer
import torch
from transformers import set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM

/scratch/miniconda3/envs/dev/lib/python3.11/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda:0")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [3]:
# lm = scorer.IncrementalLMScorer("gpt2-xl", device="cuda:0")

In [ ]:
s_full = "The nurse, who likes french cuisine, adopted a rescue dog."
s_vp1 = "The nurse likes french cuisine."
s_vp2 = "The nurse adopted a rescue dog."


def prompt(sentence):
    return f'Ellie said, "{sentence}", and Marco replied, "'


def chat_prompt(sentence, tok=tokenizer):
    return tok.apply_chat_template(
        [   {"role": "system", "content": "Please respond to the following message as naturally as possible, using a single sentence, as if we were talking to each other. Please keep it short."},
            { "role": "user", "content": sentence},
            {"role": "assistant", "content": "No, that's not true!"}],
        tokenize=False,
        continue_final_message=True
    )

In [5]:
print(chat_prompt("blah blah"))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please respond to the following message as naturally as possible, using a single sentence, as if we were talking to each other. Please keep it short.<|eot_id|><|start_header_id|>user<|end_header_id|>

blah blah<|eot_id|><|start_header_id|>assistant<|end_header_id|>

No, that's not true!


In [6]:
# encoded = lm.tokenizer(s_vp1, return_tensors="pt")
# encoded = encoded.to(lm.device)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

encoded = tokenizer([chat_prompt(s_vp1), chat_prompt(s_vp2)], return_tensors="pt", add_special_tokens=False, padding=True)
encoded = encoded.to("cuda:0")

In [7]:
set_seed(1024)

generations = model.generate(
    **encoded, 
    num_return_sequences=10,
    # max_length=100,
    max_new_tokens = 20,
    do_sample=True, 
    # top_p=None,
    # temperature=0.9,
    # typical_p=0.2,
    # # min_p=0.2,
    # # top_p=0.8,
    # # top_k = 5000,
    # force_words_ids=[[13]],
    # stop_strings=["."],
    tokenizer=tokenizer,
    top_p=None,
    temperature=0.7,
    top_k=50,
    repetition_penalty=1.2
    # length_penalty=5.0
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [8]:
tokenizer.convert_ids_to_tokens(128001)

'<|end_of_text|>'

In [9]:
input_length = encoded.input_ids.shape[-1]

In [10]:
ids = torch.arange(len(encoded.input_ids)).tolist()

ids, input_length

([0, 1], 59)

In [11]:
# generations[, [52]*10:]
# [g[52:] for g in generations.split(10)]

len(generations.split(1))

20

In [12]:
# [x for x in tokenizer.batch_decode(generations[: , input_length:].split(2), skip_special_tokens=True)]
# tokenizer.batch_decode(generations,skip_special_tokens=True)

for gen in generations[:, input_length:].split([10]*2):
    print([x for x in tokenizer.batch_decode(gen, skip_special_tokens=True)])

[" She actually loves Italian food and can't get enough of pasta carbonara.", ' She actually loves Italian food and has been trying out new pasta recipes in her free time.', ' The doctor is actually quite fond of Italian food.', ' She actually has a big soft spot for Italian food instead.', ' The doctor actually loves French food and always requests escargot when they travel to Paris.', ' The doctor is actually obsessed with sushi from Japan.', ' She actually has a weak spot for Italian food and pasta dishes.', ' The doctor is actually a huge fan of French cooking and has even written a cookbook on his favorite recipes', '', ' The doctor is more into trying new foods from around the world actually.']
[' She actually got a cat last month and named him Whiskers.', ' The vet actually adopted the cute little puppy from the shelter last weekend.', ' The vet actually adopted the pup from the shelter last weekend and named him Max.', ' She actually got her new furry friend from a reputable br

In [13]:
generations[: , input_length:].split([10]*2)[0]

tensor([[    40,   3077,   6755,   1364,    596,   8104,  21901,    315,   3623,
             80,   8065,  21016,    323,  31054,    484,    370,   2852,    325,
              0, 128009],
        [    40,   2846,    539,  14792,     11,    358,   3077,   6755,   1077,
            436,   2370,    922,    279,  14425,   1056,   1821,    520,    430,
            502,  66244],
        [    40,   3077,   6755,   1364,    596,   1027,    436,   2370,    922,
            430,    502,   8753,  66244,  19441,      0, 128009, 128001, 128001,
         128001, 128001],
        [    40,   3077,   6755,   1364,    596,   8104,  21901,    315,   3920,
            867,    354,    323,  11494,    266,    283,   4618,      0, 128009,
         128001, 128001],
        [  8100,    596,   2744,    436,   2370,    922,    279,   3920,    867,
            354,    520,    430,    502,    293,  15931,  19441,      0, 128009,
         128001, 128001],
        [  8100,    596,   2744,    436,   2370,    922,    